In [1]:
import numpy as np

import sensors
import em_stoch

In [2]:
DIST_RATIO = 0.5

In [3]:
Num_sensors1 = 5
Num_emitters1 = 1
sample_size1 = 100
theta1_rad = [0.7] # Угловые координаты источников (DoA) в радианах
theta1_deg = np.rad2deg(theta1_rad) # Угловые координаты источников (DoA) в градусах
P_1 = 1 * np.eye(Num_emitters1, dtype=np.float64) # Ковариация сигналов
Q_1 = 1.1 * np.eye(Num_sensors1, dtype=np.float64) # Ковариация шумов
A1 = (np.exp(-2j * np.pi * DIST_RATIO * np.arange(Num_sensors1).reshape(-1,1) * 
             np.sin(theta1_rad))) # Матрица векторов направленности
# Генерация сигналов, шумов и наблюдений
S1 = sensors.gss(Num_emitters1, sample_size1, P_1)
n1 = sensors.gss(Num_sensors1, sample_size1, Q_1)
X1 = (A1 @ S1.T + n1.T).T

Num_sensors2 = 5
Num_emitters2 = 2
sample_size2 = 100
theta2_rad = np.array([-0.5, 0.7], dtype=np.float64) # Угловые координаты источников (DoA) в радианах
theta2_deg = np.rad2deg(theta2_rad) # Угловые координаты источников (DoA) в градусах
P_2 = 1 * np.eye(Num_emitters2, dtype=np.float64) # Ковариация сигналов
Q_2 = 0.5 * np.eye(Num_sensors2, dtype=np.float64) # Ковариация шумов
A2 = (np.exp(-2j * np.pi * DIST_RATIO * np.arange(Num_sensors2).reshape(-1,1) * 
             np.sin(theta2_rad))) # Матрица векторов направленности
# Генерация сигналов, шумов и наблюдений
S2 = sensors.gss(Num_emitters2, sample_size2, P_2)
n2 = sensors.gss(Num_sensors2, sample_size2, Q_2)
X2 = (A2 @ S2.T + n2.T).T

In [4]:
X1_with_mv = sensors.MCAR(X1, [2,4], [0.4, 0.4])
X2_with_mv = sensors.MCAR(X2, [2,4], [0.4, 0.4])

In [5]:
X1_with_mv

array([[ 1.23286261e+00+0.9418559j , -1.85659807e-02-2.84696988j,
                    nan+0.j        ,  2.32727443e-01-0.74680402j,
                    nan+0.j        ],
       [-1.84555688e+00-0.03615284j,  5.79029362e-01+0.98613426j,
         2.52068362e-01-0.41735757j, -8.97514167e-01+0.04994758j,
                    nan+0.j        ],
       [-1.09664728e+00+1.1646421j , -1.72871621e-01-0.7934921j ,
         1.76743201e+00-1.60581369j, -1.22187416e+00+0.6652129j ,
                    nan+0.j        ],
       [-3.19705011e+00-1.03130086j,  6.19777131e-01+1.91974302j,
         2.07216841e+00-3.48689659j, -2.50062242e+00-0.98218031j,
        -3.00018978e-01+1.93779552j],
       [-1.05663395e+00-0.81800175j, -7.86290658e-01+1.43634167j,
                    nan+0.j        , -3.21607249e-01-0.98971863j,
         3.63068447e-01+1.76799945j],
       [-4.64436551e-01+1.06533348j,  4.39830960e-01+1.36970282j,
                    nan+0.j        , -1.05030240e+00+0.27738488j,
                  

In [6]:
print(np.trace(sensors.initial_Cov(X1_with_mv)))
print(np.trace(sensors.complex_cov(X1)))

(10.280945744938716+0j)
(11.105494677649398+0j)


In [7]:
EM_theta1_rad, EM_P1, lhd_1 = em_stoch.multi_start_EM(X1, Num_emitters1, Q_1, 10, 
                                                         max_iter=20, 
                                                         rtol=1e-3)

0-th start
theta=[-2.83488836],P=[2.83430482]
Initial angles = [-2.83488836]
Is SPD K_XX True
sigma=[[-33.5025035+1.77635684e-15j]]
M[X]M[X]=[[36.33677383+0.j]]
Cov_signals =[[2.83427033+1.77635684e-15j]]
likelihood is -1211.2273625699233 on iteration 0
Is SPD K_XX True
sigma=[[-33.50165357+1.77635684e-15j]]
M[X]M[X]=[[36.33588941+0.j]]
Cov_signals =[[2.83423583+1.77635684e-15j]]
likelihood is -1207.3354352920187 on iteration 1
Is SPD K_XX True
sigma=[[-32.69079724+2.22044605e-15j]]
M[X]M[X]=[[35.52499999+0.j]]
Cov_signals =[[2.83420274+2.22044605e-15j]]
likelihood is -1204.405271080364 on iteration 2
Is SPD K_XX True
sigma=[[-32.03921692+8.8817842e-16j]]
M[X]M[X]=[[34.87338771+4.4408921e-16j]]
Cov_signals =[[2.83417079+1.33226763e-15j]]
likelihood is -1202.3176150282802 on iteration 3
Is SPD K_XX True
sigma=[[-31.53745337+1.33226763e-15j]]
M[X]M[X]=[[34.37159309-4.4408921e-16j]]
Cov_signals =[[2.83413973+8.8817842e-16j]]
likelihood is -1200.8779355307418 on iteration 4
Is SPD K_XX Tru

In [8]:
print(EM_theta1_rad, EM_P1)

[0.68936891] [[2.27929762]]


In [9]:
EM_theta1_deg = np.rad2deg(EM_theta1_rad)
print(f"Погрешность в смысле разности углов в градусах: \
      {np.abs(EM_theta1_deg-theta1_deg)},")
print(f"Погрешность в смысле разности синусов углов: \
      {np.abs(np.sin(EM_theta1_rad)-np.sin(theta1_rad))}.")

Погрешность в смысле разности углов в градусах:       [0.6091163],
Погрешность в смысле разности синусов углов:       [0.00816735].


In [10]:
EM_theta2_rad, EM_P2, lhd_2 = em_stoch.multi_start_EM(X2_with_mv, Num_emitters2, Q_2, 10, 
                                                         max_iter=20, 
                                                         rtol=1e-3)

0-th start
theta=[-2.83488836  0.30670429],P=[2.83430482 3.63290896]
Initial angles = [-2.83488836  0.30670429]
Is SPD K_XX True
sigma=[[ -58.73234761+5.79839432e-15j  -32.88825634+3.03471759e+01j]
 [ -32.88825634-3.03471759e+01j -138.33944267-3.90378599e-14j]]
M[X]M[X]=[[ 48.24039259-9.22477449e-17j  21.7124367 -2.01058124e+01j]
 [ 21.7124367 +2.01058124e+01j 109.31561936-4.11854636e-16j]]
Cov_signals =[[-10.49195503+5.70614658e-15j -11.17581963+1.02413635e+01j]
 [-11.17581963-1.02413635e+01j -29.02382331-3.94497145e-14j]]
likelihood is -2100.6764710284638 on iteration 0
Is SPD K_XX True
sigma=[[-3278.74454349+1.27073487e-14j -1167.00107301+4.73190370e+01j]
 [-1167.00107301-4.73190370e+01j  -430.68095639-1.13172223e-14j]]
M[X]M[X]=[[3047.68976807-2.08546364e-14j 1096.15093162-4.45519383e+01j]
 [1096.15093162+4.45519383e+01j  395.22978276-2.02034405e-15j]]
Cov_signals =[[-231.05477542-8.14728762e-15j  -70.85014139+2.76709873e+00j]
 [ -70.85014139-2.76709873e+00j  -35.45117363-1.3337566

TypeError: unsupported operand type(s) for +: 'NoneType' and 'float'

In [ ]:
print(EM_theta2_rad, EM_P2)

[0.05921997 0.70065973] [[-1.7528317   0.        ]
 [ 0.          0.27620752]]


In [ ]:
EM_theta2_deg = np.rad2deg(EM_theta2_rad)
print(f"Погрешность в смысле разности углов в градусах: \
      {np.abs(np.sort(EM_theta2_deg)-np.sort(theta2_deg))},")
print(f"Погрешность в смысле разности синусов углов: \
      {np.abs(np.sin(EM_theta2_rad)-np.sin(theta2_rad))}.")

Погрешность в смысле разности углов в градусах:       [32.04094412  0.03779999],
Погрешность в смысле разности синусов углов:       [5.38610901e-01 5.04452319e-04].
